# Sports Savvy #

The goal of this product is to predict the performance of players in a given game. To accomplish this goal, our software will use past news and other available information and consider how similar conditions affected a player's past game. From there, it will calculate the probability of the player making his daily fantasy line.

In [2]:
import numpy as np
import pandas as pd
import requests

In [3]:
cd "C:\Users\IFE\OneDrive - eastern.edu\Documents\Fall 2023\CSCI-330B\NBA Players Stats"

[WinError 3] The system cannot find the path specified: 'C:\\Users\\IFE\\OneDrive - eastern.edu\\Documents\\Fall 2023\\CSCI-330B\\NBA Players Stats'
C:\Users\Indy.nelson\Documents


#Try 1 Importing dataset to analyze

In [4]:
player_stats_2019_2020 = pd.read_csv('2019-20_pbp.csv')
player_stats_2019_2020.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,...,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,Unnamed: 40
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,...,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,M. Gasol - gasolma01,L. Ball - balllo01,NaN
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dropping unnecessary columns and filling missing values with 0

In [5]:
# Columns to keep
keep_columns = ['Location', 'Date', 'WinningTeam', 'AwayTeam', 'HomeTeam', 'Shooter', 'ShotType', 'ShotOutcome', 'Assister', 'Rebounder', 'FreeThrowShooter', 'FreeThrowOutcome', 'TurnoverPlayer']

# Identify columns to drop
drop_columns = set(player_stats_2019_2020.columns) - set(keep_columns)

# Drop columns not in the specified set
player_stats_2019_2020.drop(columns = drop_columns, inplace = True)

player_stats_2019_2020.fillna(0, inplace = True)
player_stats_2019_2020

,Location,Date,WinningTeam,AwayTeam,HomeTeam,Shooter,ShotType,ShotOutcome,Assister,Rebounder,FreeThrowShooter,FreeThrowOutcome,TurnoverPlayer
0,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,0,0,0,0,0,0,0
1,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,L. Ball - balllo01,2-pt jump shot,miss,0,0,0,0,0
2,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,0,0,0,D. Favors - favorde01,0,0,0
3,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,D. Favors - favorde01,2-pt layup,make,0,0,0,0,0
4,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,O. Anunoby - anunoog01,2-pt layup,miss,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
539260,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,K. Olynyk - olynyke01,2-pt layup,make,0,0,0,0,0
539261,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,D. Howard - howardw01,3-pt jump shot,make,A. Caruso - carusal01,0,0,0,0
539262,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,S. Hill - hillso01,3-pt jump shot,make,0,0,0,0,0
539263,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,0,0,0,0,0,0,0,0


Transforming *ShotType* column to numeric

In [6]:
player_stats_2019_2020['ShotType'].value_counts()

ShotType
0                 336868
3-pt jump shot     78344
2-pt layup         55362
2-pt jump shot     51077
2-pt dunk          11740
2-pt hook shot      5874
Name: count, dtype: int64

In [7]:
player_stats_2019_2020['ShotType'] = player_stats_2019_2020['ShotType'].str.replace(r'-pt jump shot|-pt layup|-pt dunk|-pt hook shot','', regex=True).astype(float)
player_stats_2019_2020 = player_stats_2019_2020[(player_stats_2019_2020['ShotOutcome'] != 'miss') & (player_stats_2019_2020['FreeThrowOutcome'] != 'miss')]
player_stats_2019_2020.head()

,Location,Date,WinningTeam,AwayTeam,HomeTeam,Shooter,ShotType,ShotOutcome,Assister,Rebounder,FreeThrowShooter,FreeThrowOutcome,TurnoverPlayer
0,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,0,0,0,0
2,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,D. Favors - favorde01,0,0,0
3,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,D. Favors - favorde01,2.0,make,0,0,0,0,0
5,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,J. Redick - redicjj01,0,0,0
7,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,F. VanVleet - vanvlfr01,0,0,0


Filling missing values in *ShotType* column with 0

In [8]:
player_stats_2019_2020['ShotType'].fillna(0, inplace = True)
player_stats_2019_2020['ShotType']

C:\Users\Indy.nelson\AppData\Local\Temp\ipykernel_15312\1666426506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats_2019_2020['ShotType'].fillna(0, inplace = True)


0         0.0
2         0.0
3         2.0
5         0.0
7         0.0
         ... 
539260    2.0
539261    3.0
539262    3.0
539263    0.0
539264    0.0
Name: ShotType, Length: 417979, dtype: float64

Aggregating the player stats to create the dataframe we will work with

In [9]:
player_stats_2019_2020['Date'] = pd.to_datetime(player_stats_2019_2020['Date'])
NBA2k19_pts_df = player_stats_2019_2020.groupby(['Shooter', 'Date','AwayTeam', 'HomeTeam'])['ShotType'].sum().reset_index()
NBA2k19_pts_df = NBA2k19_pts_df[NBA2k19_pts_df['Shooter'] != 0]
NBA2k19_pts_df

C:\Users\Indy.nelson\AppData\Local\Temp\ipykernel_15312\1702485420.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats_2019_2020['Date'] = pd.to_datetime(player_stats_2019_2020['Date'])


,Shooter,Date,AwayTeam,HomeTeam,ShotType
1143,A. Aminu - aminual01,2019-10-23,CLE,ORL,3.0
1144,A. Aminu - aminual01,2019-10-28,ORL,TOR,7.0
1145,A. Aminu - aminual01,2019-10-30,NYK,ORL,8.0
1146,A. Aminu - aminual01,2019-11-01,MIL,ORL,3.0
1147,A. Aminu - aminual01,2019-11-05,ORL,OKC,8.0
...,...,...,...,...,...
22175,Z. Williamson - willizi01,2020-07-30,UTA,NOP,12.0
22176,Z. Williamson - willizi01,2020-08-01,NOP,LAC,6.0
22177,Z. Williamson - willizi01,2020-08-03,MEM,NOP,18.0
22178,Z. Williamson - willizi01,2020-08-06,NOP,SAC,20.0


In [10]:
melted_df = pd.melt(player_stats_2019_2020[['Assister', 'Rebounder', 'TurnoverPlayer', 'FreeThrowShooter', 'Date']], id_vars='Date', value_vars=['Assister', 'Rebounder', 'TurnoverPlayer', 'FreeThrowShooter'], var_name='Role', value_name='Player')

# Group by 'date', 'player', and 'role', then calculate the count of occurrences for each group
NBA2k19_rebounds_df = melted_df.groupby(['Date', 'Player', 'Role']).size().unstack(fill_value=0).reset_index()
NBA2k19_rebounds_df = NBA2k19_rebounds_df[NBA2k19_rebounds_df['Player'] != 0]
NBA2k19_rebounds_df

Role,Date,Player,Assister,FreeThrowShooter,Rebounder,TurnoverPlayer
1,2019-10-22,A. Bradley - bradlav01,0,0,3,2
2,2019-10-22,A. Davis - davisan02,5,9,9,3
3,2019-10-22,B. Ingram - ingrabr01,5,4,5,2
4,2019-10-22,D. Favors - favorde01,2,0,7,1
5,2019-10-22,D. Green - greenda02,0,1,7,0
...,...,...,...,...,...,...
23487,2020-10-11,M. Morris - morrima02,0,0,2,0
23488,2020-10-11,R. Rondo - rondora01,4,0,4,4
23489,2020-10-11,S. Hill - hillso01,1,0,0,0
23490,2020-10-11,T. Herro - herroty01,4,0,3,4


In [14]:
NBA2k19 = pd.merge(NBA2k19_pts_df, NBA2k19_rebounds_df, left_on = ['Shooter', 'Date'], right_on = ['Player', 'Date'], how = 'left')
NBA2k19['player_name'] = NBA2k19['Shooter'].combine_first(NBA2k19['Player'])
NBA2k19.drop(['Shooter', 'Player'], axis = 1, inplace = True)
NBA2k19 = NBA2k19[NBA2k19['player_name'] != 'Team']
NBA2k19.rename(columns = {'ShotType':'FG Points', 'Assister':'Assists', 'FreeThrowShooter':'FT Points', 'Rebounder':'Rebounds', 'TurnoverPlayer':'Turnovers',
                'player_name':'Player'}, inplace = True)
NBA2k19.fillna(0, inplace = True)
NBA2k19['Total Points'] = NBA2k19['FG Points'] + NBA2k19['FT Points']
NBA2k19 = NBA2k19[['Date', 'Player', 'Total Points', 'FG Points', 'FT Points', 'Rebounds', 'Assists', 'Turnovers']] # reorder columns
NBA2k19

,Date,Player,Total Points,FG Points,FT Points,Rebounds,Assists,Turnovers
0,2019-10-23,A. Aminu - aminual01,5.0,3.0,2.0,7.0,0.0,1.0
1,2019-10-28,A. Aminu - aminual01,7.0,7.0,0.0,5.0,3.0,1.0
2,2019-10-30,A. Aminu - aminual01,9.0,8.0,1.0,8.0,1.0,1.0
3,2019-11-01,A. Aminu - aminual01,3.0,3.0,0.0,5.0,1.0,2.0
4,2019-11-05,A. Aminu - aminual01,13.0,8.0,5.0,5.0,1.0,2.0
...,...,...,...,...,...,...,...,...
21032,2020-07-30,Z. Williamson - willizi01,13.0,12.0,1.0,0.0,1.0,2.0
21033,2020-08-01,Z. Williamson - willizi01,7.0,6.0,1.0,5.0,0.0,3.0
21034,2020-08-03,Z. Williamson - willizi01,23.0,18.0,5.0,7.0,5.0,2.0
21035,2020-08-06,Z. Williamson - willizi01,24.0,20.0,4.0,2.0,2.0,0.0


In [32]:
pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [1]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog

# Get all NBA players
nba_players = players.get_players()

# Create an empty list to store player game logs
player_game_logs = []

# Iterate through each NBA player
for player in nba_players:
    # Get the player ID
    player_id = player['id']
    
    # Fetch player game logs for the 2016-2017 season
    player_log = playergamelog.PlayerGameLog(player_id=player_id, season='2016-17')
    games = player_log.get_data_frames()[0]
    
    # Append player game logs to the list
    player_game_logs.append(games)

# Concatenate all player game logs into a single DataFrame
df_player_game_logs = pd.concat(player_game_logs, ignore_index=True)



df_player_game_logs.head()


ConnectionError: HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/playergamelog?DateFrom=&DateTo=&LeagueID=&PlayerID=76266&Season=2016-17&SeasonType=Regular+Season (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000022B94B3C710>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [11]:
Injury10_20 = pd.read_csv('injuries_2010-2020.csv')
Injury10_20.head()

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)


In [23]:
# Convert 'Date' column to datetime format
Injury10_20['Date'] = pd.to_datetime(Injury10_20['Date'])

# Create a boolean mask for the specified date range
mask = (Injury10_20['Date'] >= '2019-10-23') & (Injury10_20['Date'] <= '2020-06-15')

# Apply the boolean mask to select the rows within the date range
Injury1_2 = Injury10_20[mask]

# Create a list of unique player names
Injury1_2['player_name']= Injury1_2['Relinquished'].fillna('') + Injury1_2['Acquired'].fillna('')
Injury1_2['IL List']= Injury1_2['Acquired'].isnull().astype(int)

Injury1_2.drop(columns=['Acquired','Relinquished','Team','Notes'], inplace=True)

Injury1_2.head()

C:\Users\Indy.nelson\AppData\Local\Temp\ipykernel_12864\2656502157.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Injury1_2['player_name']= Injury1_2['Relinquished'].fillna('') + Injury1_2['Acquired'].fillna('')
C:\Users\Indy.nelson\AppData\Local\Temp\ipykernel_12864\2656502157.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Injury1_2['IL List']= Injury1_2['Acquired'].isnull().astype(int)
C:\Users\Indy.nelson\AppData\Local\Temp\ipykernel_12864\2656502157.py:14: SettingWithCopyWarning: 
A value is

,Date,player_name,IL List
10678,2019-10-23,Shaquille Harrison,1
10679,2019-10-23,Dylan Windler,1
10680,2019-10-23,Allen Crabbe,1
10681,2019-10-23,Chandler Parsons,1
10682,2019-10-23,Bol Bol,1


In [30]:
Injury1_2['Date'] = pd.to_datetime(Injury1_2['Date'])
NBA2k19['Date'] = pd.to_datetime(NBA2k19['Date'])

NBA2k19['player_name'] = NBA2k19['Player'].str.extract(r'(\b[A-Z]\.\s[A-Za-z]+)', expand=False)

NBA2k19_w_IL = pd.merge(NBA2k19, Injury1_2, left_on=['player_name', 'Date'], right_on=['player_name', 'Date'], how='inner')
NBA2k19_w_IL.head()

C:\Users\Indy.nelson\AppData\Local\Temp\ipykernel_12864\3989312905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Injury1_2['Date'] = pd.to_datetime(Injury1_2['Date'])


,Date,Player,Total Points,FG Points,FT Points,Rebounds,Assists,Turnovers,player_name,IL List


In [21]:
NBA2k19_w_IL['IL List'].isnull().value_counts()

Series([], Name: count, dtype: int64)

In [22]:
NBA2k19_w_IL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          0 non-null      datetime64[ns]
 1   player_name   0 non-null      object        
 2   IL List       0 non-null      int32         
 3   Player        0 non-null      object        
 4   Total Points  0 non-null      float64       
 5   FG Points     0 non-null      float64       
 6   FT Points     0 non-null      float64       
 7   Rebounds      0 non-null      float64       
 8   Assists       0 non-null      float64       
 9   Turnovers     0 non-null      float64       
dtypes: datetime64[ns](1), float64(6), int32(1), object(2)
memory usage: 132.0+ bytes


In [15]:
NBA2k19['Player'] = NBA2k19['Player'].str.split(' - ').str[0]
NBA2k19_pts_df['Shooter'] = NBA2k19['Shooter'].str.split(' - ').str[0]

NBA2k19 = pd.merge(NBA2k19_pts_df, NBA2k19, left_on = ['Shooter', 'Date'], right_on = ['Player', 'Date'], how = 'left')
NBA2k19.head()

KeyError: 'Shooter'

In [31]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonallplayers

# Get all active players
all_players = players.get_players()

# Get player IDs for the 2019-2020 season
player_ids_2019_2020 = commonallplayers.CommonAllPlayers(is_only_current_season=0, season='2019-20').get_data_frames()[0]['PERSON_ID']

# Filter the list of all players for those who played in the 2019-2020 season
players_2019_2020 = [player['full_name'] for player in all_players if player['id'] in player_ids_2019_2020]

# Create a DataFrame with a single column 'PlayerName'
df_players_2019_2020 = pd.DataFrame({'PlayerName': players_2019_2020})

# Display the resulting DataFrame
print(df_players_2019_2020.head())


            PlayerName
0   Mahmoud Abdul-Rauf
1    Tariq Abdul-Wahad
2  Shareef Abdur-Rahim
3        Michael Adams
4       Rafael Addison
